In [1]:
from pyfaidx import Sequence, Fasta

In [228]:
import tempfile
import subprocess
from io import StringIO
import pandas as pd
from shutil import which

from pathlib import Path
from typing import List, Dict, Tuple, Union, Optional

from guido.guides import Guide
from guido.helpers import rev_comp

def run_bowtie(
    guides: List[Guide],
    pam: str = 'NGG',
    core_length: int = 10,
    core_mismatches: int = 0,
    total_mismatches: int = 4,
    genome_index_path: str = '/Users/nace/imperial/guide_tool/guido/tests/data/AgamP4.12',
    threads: int = 1,
    bowtie_path: str = "bin/bowtie/",
) -> tuple:

    pam_mismatches = rev_comp(pam).count('N')
    print(pam_mismatches)
    pam_length = len(pam)

    with tempfile.NamedTemporaryFile(mode="w+t", prefix="guido_") as temp:
        temp.write(
            "\n".join(
                [(f">{i}|{G.guide_chrom}|{G.guide_start}|{G.guide_end}|"
                  f"{G.guide_seq}|{G.guide_strand}"
                  f"\n{rev_comp(G.guide_seq)}")
                  for i, G in enumerate(guides)]
            )
        )
        temp.flush()
    
        if (core_mismatches + pam_mismatches) > 3:
            raise ValueError('The value for the parameter coreMismatches is not valid: ' + str(core_mismatches))
        if core_mismatches > total_mismatches:
                raise ValueError('The value for coreMismatches cannot be greater than totalMismatches:' + str(core_mismatches) + ">" + str(total_mismatches))

    
        bowtie_command = (f"{bowtie_path}bowtie -p {threads} --quiet -y "
                        f"-n {core_mismatches + pam_mismatches} "
                        f"-l {core_length + pam_length} "
                        f"-e {(total_mismatches * 30) + 30 } -a "
                        f"-x {genome_index_path} "
                        f"-f {temp.name}")

        print(bowtie_command)
        
        rproc = subprocess.Popen(
            bowtie_command.split(),
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            universal_newlines=True,
        )
        
        stdout, stderr = rproc.communicate()

        with open('/Users/nace/imperial/guide_tool/guido.bowtie.txt', 'w') as b:
            b.write(stdout)

    return stdout, stderr

def parse_mismatches(mismatches, strand, seq_len):
    mm_split = mismatches.split(',')
    mm_dict = {}
    if len(mm_split) > 1:
        for m in mm_split:
            pos, c = m.split(':')
            c = c.split('>')

            if strand == '-':
                mm_dict[seq_len-int(pos)] = rev_comp(c[0])
            else:
                mm_dict[seq_len-int(pos)] = c[0]
    return mm_dict

def hit_is_valid(mismatches, non_arbitrary_positions):
    return all([False if pos in non_arbitrary_positions else True for pos in mismatches.keys()])

def parse_bowtie_output(out: str, pam='NGG'):

    non_arbitrary_positions = [21 + ix for ix, nucl in enumerate(pam) if nucl in ['A', 'T', 'G', 'C']]
    
    for line in out.split('\n'):
        cols = line.split('\t')
        if len(cols) > 1:
            ix, g_chrom, g_start, g_end, g_seq, g_strand = cols[0].split('|')
            t_strand, t_chrom, t_start, t_seq, _, t_rep, t_mismatches = cols[1:]
            
            if t_strand == '-':
                t_strand = '+'
            else:
                t_seq = rev_comp(t_seq)
                t_strand = '-'

            if g_chrom == t_chrom and int(g_start) == (int(t_start) + 1):
                print('------')
                print(t_seq, t_strand, cols)
                pass
            else:
                mismatches = parse_mismatches(t_mismatches, t_strand, len(t_seq))
                if hit_is_valid(mismatches, non_arbitrary_positions):
                    mm_string = ''.join([mismatches[i+1] if (i+1) in mismatches.keys() 
                                            else "." for i, _ in enumerate(g_seq)])

                    print(mm_string, hit_is_valid(mismatches, non_arbitrary_positions),)


out, err = run_bowtie_f(l.guides, core_mismatches=2, core_length=10, total_mismatches=4)
parse_bowtie_output(out)

1
bin/bowtie/bowtie -p 1 --quiet -y -n 3 -l 13 -e 150 -a -x /Users/nace/imperial/guide_tool/guido/tests/data/AgamP4.12 -f /var/folders/vk/qp7qjq952h19zhsbv1b17vc80000gn/T/guido_spj545ob
------
CCGTGCGTCCAGAGCGTTCGAGG + ['0|2R|48714149|48714172|CCGTGCGTCCAGAGCGTTCGAGG|+', '-', '2R', '48714148', 'CCGTGCGTCCAGAGCGTTCGAGG', 'IIIIIIIIIIIIIIIIIIIIIII', '0', '']
.T.G...G...A........... True
G..G....A.C.C.......... True
G..A....ATC............ True
...A.G..G...C.A........ True
.G..A.AG......T........ True
AG.......T.........TT.. True
T.CA............CC..... True
.G.....G....C.T..A..... True
...A.....A.A....A.A.... True
.......A.A.A.A....G.... True
.......A.A.TG.......G.. True
T...T......C.T......G.. True
.G...A.......T...C..G.. True
...A....A..T.....A..G.. True
.T..AA..G.........G.... True
.T..AA..G.........G.... True
.G...............C.C... True
.T..C............G.AC.. True
T..C....A.C......A..... True
...G..A.T...G....A..... True
.A...T......T.....GT... True
...AC..A.....T.....A... True
....

In [198]:
from guido.genome import Genome, load_genome_from_file
from guido.locus import load_from_coordinates, load_from_gene

chromosome, start, end = "2R", 48714143, 48714653
AgamP4 = load_genome_from_file(guido_file='tests/data/AgamP4.12.guido')

l = load_from_coordinates(AgamP4, chromosome, start, end)
l.find_guides(min_flanking_length=20, selected_features={'exon'})



In [197]:
# def check_pam_mismatches(mismatches, pam):



------
CCGTGCGTCCAGAGCGTTCGAGG +
------
GAGCAACGACCACCTGCTCTAGG +
------
TGTTTTCTGTGTGTGTGTACAGG +
------
TGTACAGGCCGTTTCTTGAACGG +
------
GTACAGGCCGTTTCTTGAACGGG +
------
TCGCCCTCCCCCCGAGACGCCGG +
------
CGCACCGCACCAGTGCAGCTAGG +
------
TCTGCTGCCCTTCTGTACCGTGG +
------
TGCCCTTCTGTACCGTGGTGCGG +
------
AAACACTAGTTTGAACTTATCGG +
------
ATCGGCATCAGTTGCGCACGCGG +
------
CACACCCACATACAAAGATACGG +
------
AAAGATACGGACAGTTACAGTGG +
------
TACGGACAGTTACAGTGGTGCGG +
------
AAGTTTATCATCCACTCTGACGG +
------
AGTTTATCATCCACTCTGACGGG +
------
TTATCATCCACTCTGACGGGTGG +
------
ATAGTAGATCCTAGAGCAGGTGG -
------
TACATAGTAGATCCTAGAGCAGG -
------
TGGTCGTCCCGTTCAAGAAACGG -
------
GAGCAGCAAAAACAAAACAGTGG -
------
AGGAAATTCTAGTGTTAAATCGG -
------
GGCGTCTCGGGGGGAGGGCGAGG -
------
CGTCCGGCGTCTCGGGGGGAGGG -
------
GCGTCCGGCGTCTCGGGGGGAGG -
------
GGTGCGTCCGGCGTCTCGGGGGG -
------
CGGTGCGTCCGGCGTCTCGGGGG -
------
GCGGTGCGTCCGGCGTCTCGGGG -
------
TGCGGTGCGTCCGGCGTCTCGGG -
------
GTGCGGTGCGTCCGGCGTCTCGG -
------
CTG

In [56]:
def isBowtieHitConsistent(bowtieLineColumns):
    '''
    Tells if the hit found show a set of mismatches, if any, consistent with the variable positions in the PAM
    ''' 
    # This generic version works for PAMs (5' or 3') where only the first position is an N
    # and the other position are not variables
    # NGG, NGA, NGCG, TTTN, NAAAAC
    # This is also fine for NRG because we search twice for NGG and NAG
    # The same for YTN (CTN, TTN)
    mismatches = bowtieLineColumns[7].split(',')
    posFirstMM = int(mismatches[0].split(":")[0])
    return posFirstMM >= (len('NGG') - 1)

out, err = run_bowtie('TTATCATCCACTCTGACGGGTGG')
for line in out.split('\n')[:]:
    columns = line.split('\t')
    if len(columns) > 1:
        if len(columns[7].split(',')) > 1:
            if isBowtieHitConsistent(columns):
                print(columns)
            else:
                print('x', columns)
        else:
            print('=', columns)    

bin/bowtie/bowtie -p 1 --quiet -y -n 1 -l 13 -e 180 -a -x /Users/nace/imperial/guide_tool/guido/tests/data/AgamP4.12 -c TTATCATCCACTCTGACGGGTGG
= ['0', '+', '2R', '48714553', 'TTATCATCCACTCTGACGGGTGG', 'IIIIIIIIIIIIIIIIIIIIIII', '0', '']
['0', '-', '2R', '28714068', 'CCACCCGTCAGAGTGGATGATAA', 'IIIIIIIIIIIIIIIIIIIIIII', '0', '15:G>T,17:T>C,18:T>C,19:G>C,20:C>A,22:T>C']
['0', '-', '3L', '17339729', 'CCACCCGTCAGAGTGGATGATAA', 'IIIIIIIIIIIIIIIIIIIIIII', '0', '13:G>A,17:G>C,18:G>C,21:A>C,22:A>C']
['0', '-', '2L', '47386269', 'CCACCCGTCAGAGTGGATGATAA', 'IIIIIIIIIIIIIIIIIIIIIII', '0', '12:C>G,16:C>G,18:G>C,20:G>A,21:T>C,22:G>C']
['0', '-', '3L', '1582363', 'CCACCCGTCAGAGTGGATGATAA', 'IIIIIIIIIIIIIIIIIIIIIII', '0', '11:T>A,14:A>C,15:C>T,18:T>C,20:T>A,21:T>C']
['0', '-', '2L', '39403975', 'CCACCCGTCAGAGTGGATGATAA', 'IIIIIIIIIIIIIIIIIIIIIII', '0', '11:T>A,13:C>A,14:G>C,15:C>T,18:T>C']
['0', '-', '2R', '39013047', 'CCACCCGTCAGAGTGGATGATAA', 'IIIIIIIIIIIIIIIIIIIIIII', '0', '11:T>A,13:T>A,14:G>C,17

In [60]:
iupac_code = {'R':'[AG]', 'Y':'[CT]', 'S':'[GC]', 'W':'[AT]', 'K':'[GT]', 'M':'[AC]', 'B':'[CGT]', 'D':'[AGT]', 'H':'[ACT]', 'V':'[ACG]', 'N':'[ACGTN]'}

def build_expression(seq):
    result = ''
    for c in seq:
        if c in iupac_code:
            result = result + iupac_code[c]
        else:
            result = result + c
            
    return result


def reverse_complement(sequence):
    rev_comp = []
    
    for idx in range(len(sequence) - 1, -1, -1):
        if sequence[idx] == 'A':
            rev_comp = rev_comp + ['T']
        elif sequence[idx] == 'C':
            rev_comp = rev_comp + ['G']
        elif sequence[idx] == 'G':
            rev_comp = rev_comp + ['C']
        elif sequence[idx] == 'T':
            rev_comp = rev_comp + ['A']
        else:
            rev_comp = rev_comp + ['N']
    return "".join(rev_comp)

class PAM(object):
    '''
    Generic class to define the different PAM types and the methods needed
    The seed for Bowtie will be always 5
    '''

    def __init__(self):
        '''
        Constructor
        '''
        self.coreAllowed = False  # By default a PAM is not compatible with the core region
        self.is5prime = False # Is the PAM at the 5' end, by default it is false
        self.PAM_str = self.__class__.__name__
        self.PAM_rev = reverse_complement(self.PAM_str)  # any ambiguous base is turned into N
        self.mismatchesPAM = self.PAM_rev.count('N')
        self.mismatchesInSeed = min(3,self.PAM_rev[0:5].count('N')) # Value for the -n parameter of Bowtie
        
    '''
    Received a list of sgRNAbindingSite objects and return a string with the
    fasta sequences formatted to perform the search of off-target sites with
    Bowtie
    '''
    def getSequencesforBowtie(self, sites):
        # By default all PAM are in the 3' end and the sequence is reversed to match the seed definition of Bowtie
        fasta = ""
        for candidate in sites:
            fasta = fasta + ">" + candidate.label + "\n"
            fasta = fasta + self.PAM_rev + reverse_complement(candidate.sequence[:-len(self.PAM_str)]) + "\n"
        return fasta
            
    def getBowtieOptions(self, coreMismatches, coreRange, totalMismatches):
        if not self.coreAllowed or coreMismatches == "NA" or coreRange == "NA":
            return ["-a", "--quiet", "-y", "-n" + str(self.mismatchesInSeed), "-l5", "-e" + str((totalMismatches + self.mismatchesPAM) * 30)]
        else:
            if (coreMismatches + self.mismatchesPAM)>3:
                raise ValueError('The value for the parameter coreMismatches is not valid: ' + str(coreMismatches))
            if coreMismatches > totalMismatches:
                raise ValueError('The value for coreMismatches cannot be greater than totalMismatches:' + str(coreMismatches) + ">" + str(totalMismatches))
            return ["-a", "--quiet", "-y", "-n" + str(coreMismatches + self.mismatchesPAM),
                    "-l" + str(coreRange + len(self.PAM_str)), "-e" + str((totalMismatches + self.mismatchesPAM) * 30)]
        
    def isBowtieHitConsistent(self, bowtieLineColumns):
        '''
        Tells if the hit found show a set of mismatches, if any, consistent with the variable positions in the PAM
        ''' 
        # This generic version works for PAMs (5' or 3') where only the first position is an N
        # and the other position are not variables
        # NGG, NGA, NGCG, TTTN, NAAAAC
        # This is also fine for NRG because we search twice for NGG and NAG
        # The same for YTN (CTN, TTN)
        mismatches = bowtieLineColumns[7].split(',')
        posFirstMM = int(mismatches[0].split(":")[0])
        return posFirstMM >= (len(self.PAM_str) - 1)

In [229]:
s = 'naceNGG'

s[:-3]

'nace'